# Hands on Quantum Computing

_Quantum Computing course, University of Helsinki, 2024_

<a href="https://colab.research.google.com/github/stfnmangini/handsonqc_helsinki2024/blob/main/HandsOnQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

Notebook to familiarize yourself with practical quantum computing and quantum software. 

**List of contents:**
1. Coding environment setup (python, notebooks, colab, qiskit, ...)  

2. Building quantum circuits  
3. Statevectors from quantum circuits  
4. Bloch sphere representation of a qubit  
5. Multi-qubit states and qiskit's ordering convetion
6. Executing a quantum circuit on a simulator
7. Executing a quantum circuit on real quantum hardware by IBM  
8. Ideal vs. noisy results
9. Noise and expectation values
10. Full example, a single-qubit VQE.

---

**...let's get started!**

# 1. Installing required packages 

Let's start by installing the necessary packages in the current environment

In [ ]:
# %pip install qiskit qiskit-aer qiskit-ibm-runtime 
# %pip install matplotlib pylatexenc
# %pip install pennylane

In [ ]:
# Import generic libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Import quantum-info library
import qiskit as qk

# 2. Create and familiarize with qiskit's quantum circuits

#### Recap of useful quantum-info facts

A qubit is a two-level quantum mechanical system whose state can be described by the so-called _statevector_:
$$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle \quad \text{with}\quad |\alpha|^2 + |\beta|^2 = 1$$

List of common single and two-qubit gates:

<img src="imgs/single_qubit_gates.png" width = "750px"/>
<div>
<img src="imgs/two_qubit_gates.png" width= "750px"/>

In [ ]:
# Create a quantum circuit with 1 qubit
num_qubits = 1
qc = qk.QuantumCircuit(num_qubits)

# Draw a graphical representation of the circuit (using matplotlib)
print(qc) # qc.draw(output='mpl')

In [ ]:
# Start same as before, but with adding more gates
num_qubits = 1
qc = qk.QuantumCircuit(num_qubits)

# Add a Hadamard gate
qc.h(0)

# add other gates...

# add a measurement gate...

# Draw a graphical representation of the circuit (using matplotlib)
print(qc) # qc.draw(output='mpl')

#### More qubits, more fun

In [ ]:
# Start same as before, but with more qubits
num_qubits = 4
qc = qk.QuantumCircuit(num_qubits)

# Add a layer of single qubit gates
for n in range(num_qubits):
    qc.h(n)

# Add a layer of entangling gates
control_qubit_index = 0
target_qubit_index = 1
qc.cx(control_qubit_index, target_qubit_index)

# Draw a graphical representation of the circuit (using matplotlib)
qc.draw(output='mpl')

# 3. Let's get the statevectors

Useful reminder: the state of a single-qubit system is described by a pair of complex numbers

$$|\psi\rangle = c_0 |0\rangle + c_1 |1\rangle \longrightarrow [c_0,\, c_1]$$

In [ ]:
from qiskit.quantum_info import Statevector
? Statevector

In [ ]:
qc = qk.QuantumCircuit(1)

# add gates...

# Let qiskit compute the statevector of corresponding to the quantum circuit
psi = Statevector(qc)
print("|psi> = ", psi.data)

# 4. Boch sphere representation of a qubit

In [ ]:
from qiskit.visualization import plot_bloch_multivector

# Create a quantum circuit with 1 qubit and apply a gate
qc = qk.QuantumCircuit(1)
qc.x(0)

# Plot its representation in the bloch sphere
plot_bloch_multivector(qc)

In [ ]:
def bloch_vector(state):
    """
    Calculate components in Bloch Sphere given a quantm state (a ket |psi>) 
    """
    I = np.array([[1, 0], [0, 1]])
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])

    rho = np.outer(state, np.conjugate(state))
    rx = np.trace(np.dot(rho, X)).real
    ry = np.trace(np.dot(rho, Y)).real
    rz = np.trace(np.dot(rho, Z)).real
    return rx, ry, rz

def plot_state(states, ax = None):
    """
    Plot single qubit quantum states on the bloch sphere.
    """
    from qiskit.visualization import plot_bloch_vector

    states = np.array(states)
    
    if ax == None:
        fig = plt.figure(figsize=(9.6, 6))
        ax = plt.subplot(projection='3d')

    plot_bloch_vector([0, 0, 0], ax=ax)

    if states.ndim == 1:
        states = np.expand_dims(states, 0)

    cmap = plt.cm.viridis.resampled(len(states))

    for i, state in enumerate(states):
        x, y, z = bloch_vector(state)
        ax.plot(y, -x, z, marker='o', color=cmap(i), markersize=10, alpha=1.)
    return ax

In [ ]:
# Visualizing rotations along the Pauli axis
theta = np.linspace(0, np.pi, 10)

state_list = []
for t in theta:
    qc = qk.QuantumCircuit(1)
    qc.rx(t, 0)
    state_list.append(Statevector(qc).data)    

plot_state(state_list)

# 5. Statevectors of multi-qubit systems

In [ ]:
qc = qk.QuantumCircuit(2)
psi = Statevector(qc)
psi

In [ ]:
qc = qk.QuantumCircuit(2)
psi0 = Statevector(qc)
print("Initial state: |0> ⊗ |0>  = ", psi0.data)

qc.x(0)
psi = Statevector(qc)
print("Final state:  X|0> ⊗ I|0> = ", psi.data)

_Mind qiskit's ordering!_

When we have two qubits, a generic state is defined as
$$ 
\begin{align} 
|\psi\rangle & = c_0 |0\rangle_{q_0}|0\rangle_{q_1} + c_1 |0\rangle_{q_0}|1\rangle_{q_1} + c_2 |1\rangle_{q_0}|0\rangle_{q_1} + c_3 |1\rangle_{q_0}|1\rangle_{q_1} \\
& = c_0 |00\rangle + c_1 |01\rangle + c_2 |10\rangle + c_3 |11\rangle \\
& = c_0 \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix} + c_0 \begin{bmatrix} 0 \\ 1 \\ 0 \\ 0 \end{bmatrix} + c_0 \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix} + c_0 \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix} = [c_0,\, c_1,\, c_2,\, c_3]
\end{align}
$$

Starting from the initial ground state, the circuit implements
$$|\psi\rangle = |0\rangle_{q_0}|0\rangle_{q_1} = |0\rangle_{q_0} \otimes |0\rangle_{q_1} \xrightarrow[X \otimes \mathbb{I}]{\text{Evolve with gates}} |\psi'\rangle = X|0\rangle_{q_0}\otimes \mathbb{I}|0\rangle_{q_1} = |1\rangle_{q_0}|0\rangle_{q_1}$$

so, in coefficients we would have $$|\psi\rangle = [1,\,0,\,0,\,0] \longrightarrow |\psi'\rangle = [0,\, 0,\, 1,\, 0]$$

But qiskit orders qubit indices differently, in reversed (_"little-endian"_) order, that is 
$$ |\psi\rangle_{\text{qiskit}} = c_0 |0\rangle_{q_1}|0\rangle_{q_0} + c_1 |0\rangle_{q_1}|1\rangle_{q_0} + c_2 |1\rangle_{q_1}|0\rangle_{q_0} + c_3 |1\rangle_{q_1}|1\rangle_{q_0} $$

so the circuit acts as
$$|0_{q_1}0_{q_0}\rangle \longrightarrow \mathbb{I} |0\rangle_{q_1} \otimes \mathbb{X}|0\rangle_{q_0} = |0_{q_1}1_{q_0}\rangle$$

hence the final state is given by
$$|\psi\rangle_{\text{qiskit}} = [1,\,0,\,0,\,0] \longrightarrow |\psi'\rangle_{\text{qiskit}} = [0,\, 1,\, 0,\, 0]$$


# 6. Finally running a quantum circuit

In [ ]:
# Impprt the AerSimulator backend from qiskit
from qiskit_aer import AerSimulator
sim_bknd = AerSimulator()

# Define a quantum circuit with 2 qubits and a measurement gate
qc = qk.QuantumCircuit(2)
qc.measure_all()

# Run the circuit on the AerSimulator backend anf get the results
num_shots = 1
res = sim_bknd.run(qc, shots = 1).result()

# Print the results
print("Measurement outcomes\n", res.get_counts())

qc.draw(output='mpl')

In [ ]:
? sim_bknd.run

# 7. Running on an actual quantum deivce (sort-of)

In [ ]:
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService

# You have to be logged in to IBM Quantum to run on real hardware.
# You can do this by registering on IBMQ and running the following commands
# ibm_quantum_service = QiskitRuntimeService(channel="ibm_quantum", token="MY_IBM_QUANTUM_TOKEN")
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="MY_IBM_QUANTUM_TOKEN")

# Once you have done it, you can run the following code
service = QiskitRuntimeService()
bknd_list = service.backends()
bknd_list

In [ ]:
# Pick the least busy "real" backend
bknd = service.least_busy(simulator=False, operational=True)
bknd.name

In [ ]:
from qiskit.visualization import plot_gate_map, plot_error_map, plot_circuit_layout

plot_gate_map(service.get_backend('ibm_osaka'),
              figsize=(10, 10), 
              font_size=20, 
              line_width=2, 
              qubit_size=50,
              line_color = ['black'] * len(bknd.coupling_map.get_edges()),
              qubit_color = [mpl.colors.to_hex(c) for c in plt.cm.viridis_r(np.linspace(0, 1, bknd.num_qubits))])

In [ ]:
# Define a quantum circuit with 5 qubits and a measurement gate
qc = qk.QuantumCircuit(5)
qc.measure_all()
print(qc)

# Run the circuit on the real quantum hardware
# simple_run = bknd.run(qc, shots=1000)
# print(f">>> Job ID: {simple_run.job_id()}")
# print(f">>> Job Status: {simple_run.status()}")

In [ ]:
# This is the new and correct way of running on real hardware
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Preprocess the circuit so that it is compatible with the backend
pm = generate_preset_pass_manager(optimization_level=0, backend=bknd)
# Obtain the Instruction Set Architecture (ISA) circuit
isa_circuit = pm.run(qc)

# Use the Sampler class to run the experiment
sampler = Sampler(backend=bknd)
# job = sampler.run([isa_circuit], shots=1000)
# print(f">>> Job ID: {job.job_id()}")
# print(f">>> Job Status: {job.status()}")

# Get the counts with (0 because we only have one circuit)
# counts = job.result()[0].get_counts()
# counts

In [ ]:
# Retrieve the results from a previous job
real_results = service.job('cs151tzyhpyg008agnsg')

# Print the backend used
print(real_results.backend())

# Print the data of the job
for k, v in real_results.metrics().items():
    print(f"{k}: {v}")

# Print the quantum circuit used
real_results.inputs['circuits'][0].draw(output='mpl')

In [ ]:
real_counts = real_results.result().get_counts()
real_counts

In [ ]:
sim_bknd = AerSimulator()
sim_counts = sim_bknd.run(qc, shots=1000).result().get_counts()
sim_counts

In [ ]:
# Define a fake simulator backend with the same properties as the real backend
fake_qc = AerSimulator().from_backend(real_results.backend())
fake_counts = fake_qc.run(qc, shots=1000).result().get_counts()

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram([sim_counts, fake_counts, real_counts], 
               legend=['Simulator', 'Fake QC', 'Real QC'])

#### Same, but different: using entangled states

In [ ]:
qc_ghz = qk.QuantumCircuit(5)

qc_ghz.h(0)
for i in range(0, 4):
    qc_ghz.cx(i, i+1)

qc_ghz.measure_all()
qc_ghz.draw(output='mpl')

In [ ]:
qc_ghz_transpiled = qk.transpile(qc_ghz, backend=bknd)
qc_ghz_transpiled.draw(output='mpl')

In [ ]:
plot_circuit_layout(qc_ghz_transpiled, bknd)

In [ ]:
# Retrieve previously run job with GHZ
ghz_run = service.job('cs17p89kfpw00080d2zg')
ghz_run.status()
real_ghz_counts = ghz_run.result().get_counts()

# Simulate ideal experiment with GHZ
sim_ghz_counts = sim_bknd.run(qc_ghz, shots=1000).result().get_counts()
plot_histogram([real_ghz_counts, sim_ghz_counts],
               legend=['Real QC', 'Simulator'])

# 9. Let's go noisy, with a detour on expectation values

In [ ]:
qc = qk.QuantumCircuit(1)
qc.x(0)

# ...what if there is an additional error rotation

plot_state(Statevector(qc).data)

#### One does ~~not~~ simply compute expectation values

$$Z = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix} = |0\rangle\!\langle 0| - |1\rangle\!\langle1|$$

$$\langle Z \rangle = \text{Tr}[Z\,\rho] = 1 \times \text{Tr}[\rho\, |0\rangle\!\langle 0|] - 1 \times \text{Tr}[\rho\, |1\rangle\!\langle1|] = 1 \times p_0 -1 \times p_1

In [ ]:
def expval_from_counts(outcomes_dict):
    exp_val = 0
    for outcome, counts in outcomes_dict.items():
        val = 1 if outcome == '0' else -1
        exp_val += val * counts
    num_shots = sum(outcomes_dict.values())
    return exp_val / num_shots

qc = qk.QuantumCircuit(1)
qc.rx(0.3, 0)
qc.measure_all()

res = sim_bknd.run(qc, shots=1000).result()
counts = res.get_counts()

print("Measurement outcomes:", counts)
print("<Z> = 1 * Freq(0) - 1 * Freq(1) =", expval_from_counts(counts))

qc.remove_final_measurements()
plot_state(Statevector(qc).data);

In [ ]:
# Same way, but using qiskit functions
from qiskit.quantum_info import Pauli

qc = qk.QuantumCircuit(1)
qc.rx(0.3, 0)
pauliZ = Pauli('Z')
qk_expval = Statevector(qc).expectation_value(pauliZ)
print("Qiskit expectation value: ", qk_expval)

#### Computing X and Y expectation values by change of basis

$$ X = H~Z~H $$
$$ Y = (S H)~Z~(H S^\dagger)

In [ ]:
qc = qk.QuantumCircuit(1)
qc.rx(0., 0)

# Define a circuit including a change of basis circuit
qc_meas = qc.copy()
measure = 'Z'
match measure:
    case 'Z':
        pass
    case 'X':
        qc_meas.h(0)
    case 'Y':
        qc_meas.sdg(0)
        qc_meas.h(0)

# Measure in the X basis
qc_meas.measure_all()

counts = sim_bknd.run(qc_meas, shots=1000).result().get_counts()
print("Measurement outcomes:", counts)
print("<Z> = 1 * Freq(0) - 1 * Freq(1) =", expval_from_counts(counts))

plot_state(Statevector(qc).data);

#### Back to noise!

Let's assume that every $X$ gate is faulty, that is we experimentally implement

$$X \longrightarrow X_{\text{noisy}} =  R_{X}(\varepsilon) X $$

In [ ]:
from qiskit_aer.noise import NoiseModel, coherent_unitary_error

def pauliX_rotation(theta):
    """Function that returns the unitary for a Pauli-X rotation gate"""
    op = np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], 
                   [-1j*np.sin(theta/2), np.cos(theta/2)]]) 
    return op

eps = 0.05
error = coherent_unitary_error(pauliX_rotation(eps))

In [ ]:
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
print(noise_model)

In [ ]:
noisy_bknd = AerSimulator(noise_model=noise_model)

qc = qk.QuantumCircuit(1)
qc.x(0)
qc.measure_all()

qc_transpiled = qk.transpile(qc, backend=noisy_bknd)

counts = noisy_bknd.run(qc_transpiled, shots = 10000).result().get_counts()
print("Measurement outcomes:", counts)
expval = expval_from_counts(counts)
print("Expectation value:", expval)

qc_transpiled.draw(output='mpl')

**What if we have more gates?**

In [ ]:
qc_list = []
expval_list = []
for d in range(100):
    qc = qk.QuantumCircuit(1)
    for _ in range(d):
        qc.x(0)
        qc.x(0)
    qc.measure_all()

    qc_transpiled = qk.transpile(qc, backend=noisy_bknd, optimization_level=0)
    counts = noisy_bknd.run(qc_transpiled, shots = 1000).result().get_counts()
    expval = expval_from_counts(counts)

    qc_list.append(qc_transpiled)
    expval_list.append(expval)

qc_list[2].draw(output='mpl')

In [ ]:
plt.plot(expval_list, marker='o', ms = 4, mec = 'k', ls = '--', label = "Noisy")
plt.hlines(1, 0, len(expval_list), color='g', ls='-.', label = "Ideal")

plt.legend()
plt.xlabel('Number of XX = Id gates')
plt.ylabel(r'Expectation value $\langle{Z}\rangle$')

#### Let's put it all together

In [ ]:
def run_noisy_exp(eps):
    """
    """
    print("Running experiments with epsilon =", eps)

    # Define the error model and create noisy backend
    error = coherent_unitary_error(pauliX_rotation(eps))
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    noisy_bknd = AerSimulator(noise_model=noise_model)

    # Run the experiments for different depths
    qc_list = []
    expval_list = []
    for depth in range(100):
        qc = qk.QuantumCircuit(1)
        for d in range(depth):
            qc.x(0)
            qc.x(0)
        qc.measure_all()

        qc_transpiled = qk.transpile(qc, backend=noisy_bknd, optimization_level=0)
        counts = noisy_bknd.run(qc_transpiled, shots = 1000).result().get_counts()
        expval = expval_from_counts(counts)

        qc_list.append(qc_transpiled)
        expval_list.append(expval)

    return expval_list

In [ ]:
error_levels = [0, 0.01, 0.05, 0.1]
res = [run_noisy_exp(e) for e in error_levels]

colors = plt.cm.Set1((0,1,2,3))
for x, e, c in zip(res, error_levels, colors):
    plt.plot(x, label = fr'$\epsilon = {e}$', marker='o', ms = 4, mec = 'k', ls = '--', c=c)

plt.xlabel('Number of XX = Id gates')
plt.ylabel(r'Expectation value $\langle{Z}\rangle$')
plt.legend()

#### Incoherent noise model

In [ ]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

qc = qk.QuantumCircuit(4)
qc.cx(0,1)
qc.cx(2,3)
qc.measure_all()
qc.draw(output='mpl')

In [ ]:
error = depolarizing_error(0.1, 2)

noise_model = NoiseModel()
noise_model.add_quantum_error(error, ['cx'], [0, 1])
noise_model.add_quantum_error(error, ['cx'], [2, 3])
noisy_bknd = AerSimulator(noise_model=noise_model)
print(noise_model)
        
qc_transpiled = qk.transpile(qc, noisy_bknd)
counts = noisy_bknd.run(qc_transpiled).result().get_counts()
plot_histogram(counts)

In [ ]:
# Can you guess what this function is doing
sum([freq * (-1) ** sum([int(x) for x in bitstring]) for bitstring, freq in counts.items()]) / sum(counts.values())

In [ ]:
# Doing the same with Qiskit functions
from qiskit.quantum_info import DensityMatrix

qc_nomeas = qc.copy()
qc_nomeas.remove_final_measurements()
qc_nomeas.save_density_matrix()
qc_nomeas.draw()

rho = noisy_bknd.run(qc_nomeas).result().data()['density_matrix']
rho.expectation_value(Pauli('ZZZZ'))

# 10. An example of VQAs with Pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device('default.qubit', wires=1)

In [ ]:
@qml.qnode(dev)
def circuitRX(theta):
    qml.RX(theta, wires=0)
    return qml.state()

# Choose a rotation angle
theta = np.pi/2

# Plot quantum circuit
qml.draw_mpl(circuitRX)(0.5)

# Action in the bloch sphere representation
state = circuitRX(theta).numpy()
plot_state(state)

print(f"Rx rotation with θ = {theta}")
print("|psi> = ", state)

#### Optimization: a complete example
Let's now build a complete example: automatically move the qubit from a starting excited state to the low energy state $|0\rangle$. The "energy" of quantum system is encoded in a matrix called _Hamiltonian_, denoted with $\mathcal{H}$. In our case, we consider an Hamiltonian $\mathcal{H}=-Z$, where $Z$ is the Pauli matrix:
$$
H = -Z = -\begin{bmatrix}1 & 0 \\ 0 & -1 \end{bmatrix}\,,\quad |0\rangle = \begin{bmatrix}1 \\ 0 \end{bmatrix}\,,\quad |1\rangle = \begin{bmatrix}0 \\ 1 \end{bmatrix}
$$
The energies of the single qubit _ground state_ $|0\rangle$ and the excited state $|1\rangle$ are then
$$
\text{Energy of state}\,\, |0\rangle = E_{0} = \langle 0 | H | 0 \rangle = -1
$$

$$
\text{Energy of state}\,\, |1\rangle = E_{1} = \langle 1 | H | 1 \rangle = 1
$$

We will now find a way to minimize the energy of a qubit automatically by gradient descent!  

First, define a general circuit that uses rotations `RX` and `RY` sequentially:

In [ ]:
@qml.qnode(dev)
def energy_circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(-1.*qml.PauliZ(0)) # <--- This is like an energy term, something that we want to minimize

@qml.qnode(dev)
def state_circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.state()

qml.draw_mpl(energy_circuit)([0,0,0]);

# Let's first generate three random initial parameters
thetas = np.random.rand(2)
e = energy_circuit(thetas)

print(f"θ = {thetas}")
print(f"Energy(θ) = {e}")

With this circuit, the energy function that we are trying to minimize is 

$$
|\psi\rangle = R_x(\theta)R_y(\phi)|0\rangle \implies \text{Energy of}\, |\psi\rangle\, \text{is}:\quad E_{|\psi\rangle}(\theta,\phi) :\,=\langle \psi | H | \psi \rangle = -\cos(\theta)\cos(\phi)
$$

Define an _optimization method_, in this case we use plain Gradient Descent:

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.2)

In [ ]:
def training_circuit(cost_function, params, **kwargs):
    """
    Optimize a cost function given some initial parameters
    """

    # Store the values of the cost function
    energy = [cost_function(params, **kwargs)]

    # Store the parameters of the circuit (the rotation angles)
    angle = [params]

    # Set some stopping criterion for the optimization
    max_iterations = 100
    conv_tol = 1e-08

    # Optimization
    for n in range(max_iterations):
        # Perform a gradient descent step and obtain new parameters in the circuit
        params, prev_energy = opt.step_and_cost(cost_function, params, **kwargs)

        # Save new energy and new parameters 
        energy.append(cost_function(params, **kwargs))
        angle.append(params)

        # Check if reached convergence
        conv = np.abs(energy[-1] - prev_energy) 

        # Print intermediate values
        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f}")

        # Break if convergence tolerance is reached
        if conv <= conv_tol:
            print("Convergence reached.")
            break

    print(f"\nFinal value of the ground-state energy = {energy[-1]:.8f}")
    print(f"Optimal value of the circuit parameter = {np.round(angle[-1], 8)}")

    return energy, np.array(angle)

initial_parameters = np.array([.23, 2.4], requires_grad=True)
energy_history, params_history = training_circuit(energy_circuit, initial_parameters)

In [ ]:
fig = plt.figure(figsize=(20, 5))

ax1 = plt.subplot2grid((2, 4), (0, 0), colspan=1, rowspan=2)
ax2 = plt.subplot2grid((2, 4), (0, 1))
ax3 = plt.subplot2grid((2, 4), (1, 1))
ax4 = plt.subplot2grid((2, 4), (0, 2), projection = '3d', rowspan=2)
ax5 = plt.subplot2grid((2, 4), (0, 3), rowspan=2)

ax1.set_title("Energy history", fontsize=20)
ax1.set_xlabel("Optimization step", fontsize=12)
ax1.set_ylabel(r"Energy function $\langle H \rangle$", fontsize=12)
ax1.hlines(-1, 0, len(energy_history), lw=2, ls='--', color=plt.cm.coolwarm(0), label="True $E_{min}$")
ax1.plot(energy_history, ls="--", marker='o', color=plt.cm.coolwarm_r(0), label="$E_{circuit}$")
ax1.legend(fontsize = 18)

ax2.set_title("Parameters history", fontsize=20)
ax2.set_xticks([])
ax2.set_ylabel("θ1", fontsize=12)
ax2.plot(params_history[:, 0])
ax2.set_ylim(0,np.pi)

ax3.set_xlabel("Optimization Step", fontsize=12)
ax3.set_ylabel("θ2", fontsize=12)
ax3.plot(params_history[:, 1])
ax3.set_ylim(0, np.pi)

states = np.array([state_circuit(p) for p in params_history])
plot_state(states, ax=ax4)

ax5.set_title("Energy landscape", fontsize = 20)
ax5.set_xlabel("θ1", fontsize=12)
ax5.set_ylabel("θ2", fontsize=12)
x = np.linspace(-np.pi, np.pi, 100)
y = np.linspace(-np.pi, np.pi, 100)
x, y = np.meshgrid(x, y)
z = -np.cos(x)*np.cos(y)
cnt = ax5.contourf(x, y, z, 100, levels=np.arange(-1, 1.1, 0.1), cmap=plt.cm.coolwarm, alpha=0.8, extend="both")
ax5.scatter(params_history[-1, 0], params_history[-1, 1], color='r', marker='X', s=200, zorder=2)
ax5.plot(params_history[:, 0], params_history[:, 1], ls='--', marker='.', c='y', markersize=10, zorder=1)
plt.colorbar(cnt, ticks=[-1, 0, 1])

plt.tight_layout()
plt.show()

#### _...the end._